In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math
from flows.models import VARmodel

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# device_lib.list_local_devices()

In [3]:
!ls CDATA

AUS.csv  FRA.csv  GBR.csv  SYNTH.csv


In [4]:
ccodes = ['FRA']#, 'FRA', 'GBR']
datas = ['./CDATA/{}.csv'.format(x) for x in ccodes]

In [5]:
datas = [pd.read_csv(x, index_col='VARIABLE').values.astype(floatX).T[np.newaxis][:,1:-1] for x in datas]

In [6]:
[data.shape for data in datas]

[(1, 193, 4)]

In [7]:
scaler = 0.
for i, data in enumerate(datas):
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    scaler = scaler + stds
    datas[i] = data
print('---')
scaler /= len(datas)
for i in range(len(datas)):
    datas[i] /= scaler
    data = datas[i]
    stds = (data[0,1:] - data[0,:-1]).std(axis=0)
    print(stds)
    data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
    datas[i] = data

[0.00571698 0.93158025 0.00503056 0.03016526]
---
[0.99999994 0.99999994 0.99999934 1.        ]


In [8]:
class VARmodel:
    def __init__(self, data, name='VARmodel', var_dim=None, mu=None, current_year=None):
        self.num_samples = 100
        self.data_raw = data
        self.mu = mu
        self.var_dim = var_dim
        self.years = data.columns.values.astype('float32')
        years_c = tf.constant(data.columns.values, dtype=tf.float32, name='data_years')

        if current_year is None:
            self.OBSERV_STEPS = np.Infinity
        else:
            self.OBSERV_STEPS = tf.reduce_sum(tf.cast(years_c <= current_year, tf.int32))

        self.NUM_STEPS = data.shape[1]
        self.name = name
        self.logdensities = []
        self.priors = []
        self.dim = [self.var_dim,self.var_dim*2+1]
        self.summaries = []

        self.observable_mask = tf.range(0, self.NUM_STEPS, dtype=tf.int32) < self.OBSERV_STEPS

        pd = np.std(data.values[:,1:] - data.values[:,:-1], axis=-1).astype(floatX)[:self.var_dim]

        with tf.variable_scope(name, dtype=floatX) as scope:
            self.data = tf.get_variable(initializer=data.values.T[np.newaxis].astype(floatX),
                                    trainable=False, name='data')
            self.scope = scope

            self.create_rw_priors()
            self.outputs = self.create_walk_inference(mu=mu)
            self.create_observ_dispersion_inference(pd*0.5)
            self.create_likelihood(self.observable_mask, self.outputs)

    def create_summary(self, stype, name, tensor):
        s = stype(name, tensor)
        self.summaries.append(s)

    def create_rw_priors(self):
        dim = self.dim
        with tf.variable_scope('rw_priors'):
            s1 = 0.01/4
            cov_prior = Normal(dim=None, mu=0.5*math.log(1/s1), sigma=3.5, name='cov_prior')

            with tf.variable_scope('PWalk_inf'):
                with tf.variable_scope('flows'):
                    flow_conf = [NVPFlow(dim=self.dim[0]*self.dim[1], name='nvp_{}'.format(i)) for i in range(4)] + \
                        [LinearChol(dim=self.dim[0]*self.dim[1], name='lc')]
                    ldiag = DFlow(flow_conf, num_samples=self.num_samples)
                    print(ldiag.logdens)
                    ldiag.output += 0.5*math.log(1/s1)
                    ldiag.logdens -= tf.reduce_sum(ldiag.output, axis=-1)
                    print('ldiag logdens', ldiag.logdens)

                    self.logdensities.append(ldiag.logdens)
                
                if self.mu is None:
                    sigma0 = None
                else:
                    sigma0 = 3.
                    
                PWalk = MVNormalRW(dim=self.dim[0]*self.dim[1], 
                                   sigma0=3., 
                                   diag=tf.exp(ldiag.output), name='OrdWalk')
                self.priors.append(tf.reduce_sum(cov_prior.logdens(ldiag.output, reduce=False), axis=1))
                self.PWalk = PWalk
                tf.summary.scalar('s1_ord', tf.reduce_mean(tf.sqrt(PWalk.diag)))

    def create_walk_inference(self, mu=None):
        dim = self.dim
        gvar = GVAR(dim=dim[0]*dim[1], len=self.NUM_STEPS, name='coef_rw_inference', mu=mu, num_samples=self.num_samples)
        outputs = gvar.sample()

        self.logdensities.append(gvar.logdens)
        with tf.name_scope('PWald_prior'):
            self.priors.append(tf.reduce_sum(self.PWalk.logdens(outputs), axis=[1]))
        self.outputs = outputs

        return outputs

    def create_observ_dispersion_inference(self, prior_disp):
        print('Prior disp: {}'.format(prior_disp))
        prior_loc = -0.5*np.log(prior_disp).astype(floatX)

        with tf.variable_scope('obs_d_inf', reuse=tf.AUTO_REUSE):
            flow_conf = [NVPFlow(dim=self.var_dim, name='nvp_{}'.format(i)) for i in range(6)] + \
                [LinearChol(dim=self.var_dim, name='lc')]
            ldiag = DFlow(flow_conf, init_sigma=0.05, num_samples=self.num_samples)

            ldiag.output -= prior_loc
            ldiag.logdens -= tf.reduce_sum(ldiag.output, axis=-1)

        self.obs_d = tf.distributions.Normal(tf.constant(0., dtype=floatX), 
                                             scale=tf.exp(ldiag.output), name='obs_d_prior')
        
        with tf.name_scope('obsrv_prior'):
            pr = tf.reduce_sum(tf.distributions.Normal(loc=prior_loc[np.newaxis], 
                                                       scale=tf.constant(3., dtype=floatX)).log_prob(ldiag.output), 
                               axis=-1)
        tf.summary.scalar('mean_ods', tf.reduce_mean(tf.exp(ldiag.output)))
        self.logdensities.append(ldiag.logdens)
        self.priors.append(pr)

    def predict(self, observable_mask, outputs):
        dim = self.dim
        data = self.data
        out = tf.reshape(outputs, [self.num_samples, self.NUM_STEPS, dim[0], dim[1]])
        out = tf.transpose(out, [1,0,2,3])

        def step(prev, x):
            mask = x[0]
            prev_pred = tf.where(mask, x[1], prev)
            params = x[2]

            d0 = params[:,:,:dim[0]]
            d1 = params[:,:,dim[0]:2*dim[0]]

            pp1 = prev_pred[:,:dim[0]]
            pp0 = prev_pred[:,dim[0]:2*dim[0]]

            new_pred = tf.einsum('bij,bj->bi', d0, pp0) + tf.einsum('bij,bj->bi', d1, pp1)+ params[:,:,-1] + pp1
            obs_noise = self.obs_d.sample()
            new_pred = tf.where(mask, new_pred, new_pred + obs_noise)

            new_pred = tf.concat([new_pred, pp1], axis=1)
            return new_pred
        
        data = tf.transpose(tf.tile(data, [self.num_samples, 1, 1]), [1,0,2])
        ar = tf.scan(step, [observable_mask, data, out], 
                     initializer=tf.zeros([self.num_samples, 2*dim[0]], dtype=floatX))
        return ar

    def create_likelihood(self, observable_mask, outputs):
        dim = self.dim
        obs_d = self.obs_d

        preds = self.predict(observable_mask, outputs)
        self.preds = preds[:,:self.var_dim]
        
        with tf.name_scope('loglikelihood'):
            data = tf.transpose(self.data, [1,0,2])
            diffs = preds[:-1,:] - data[1:,:]
            diffs = diffs[:,:,:dim[0]]

            logl = tf.reduce_sum(obs_d.log_prob(diffs), [2])
            print(logl)
            logl *= tf.cast(self.observable_mask[1:], floatX)[:,tf.newaxis]

            logl = tf.reduce_sum(logl, axis=0)
            self.priors.append(logl)


In [9]:
country_data = {c:d for c,d in zip(ccodes, datas)}

In [10]:
datas[0].shape

(1, 192, 8)

In [11]:
d = datas[0][0]
d = pd.DataFrame(d.T, columns=range(d.shape[0]))
datas[0] = d

In [12]:
device = '/cpu:0'

In [13]:
# with tf.device(device):
with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
    model = VARmodel(datas[0], name='SYNTH', var_dim=4, current_year=3000.)

Tensor("model/SYNTH/rw_priors/PWalk_inf/flows/sub:0", shape=(100,), dtype=float32)
ldiag logdens Tensor("model/SYNTH/rw_priors/PWalk_inf/flows/sub_1:0", shape=(100,), dtype=float32)
Prior disp: [0.49601343 0.5011316  0.4979925  0.5013071 ]
Tensor("model/SYNTH/loglikelihood/Sum:0", shape=(191, 100), dtype=float32)


In [14]:
model.priors

[<tf.Tensor 'model/SYNTH/rw_priors/PWalk_inf/Sum:0' shape=(100,) dtype=float32>,
 <tf.Tensor 'model/SYNTH/PWald_prior/Sum:0' shape=(100,) dtype=float32>,
 <tf.Tensor 'model/SYNTH/obsrv_prior/Sum:0' shape=(100,) dtype=float32>,
 <tf.Tensor 'model/SYNTH/loglikelihood/Sum_1:0' shape=(100,) dtype=float32>]

In [15]:
model.logdensities

[<tf.Tensor 'model/SYNTH/rw_priors/PWalk_inf/flows/sub_1:0' shape=(100,) dtype=float32>,
 <tf.Tensor 'model/SYNTH/coef_rw_inference_1/VAR/logdens/add:0' shape=(100,) dtype=float32>,
 <tf.Tensor 'model/SYNTH/obs_d_inf/sub_2:0' shape=(100,) dtype=float32>]

In [16]:
# with tf.device(device):
prior = tf.add_n(model.priors)

logdensity = tf.add_n(model.logdensities)
kl = logdensity - prior
kl = tf.reduce_mean(kl)
kl /= 36*160

main_op = tf.train.AdamOptimizer(0.001).minimize(kl)

In [17]:
kls = tf.summary.scalar('KLd', kl)
summary = tf.summary.merge_all()

In [18]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [19]:
init.run()

In [20]:
writer = tf.summary.FileWriter('/home/nikita/tmp/tblogs/multisample100_float32')

In [21]:
writer.add_graph(tf.get_default_graph())

In [22]:
epoch = 0

In [ ]:
for epoch in range(epoch, 100000):
    print(epoch)
    for step in range(100):
        sess.run(main_op)
    s, _ = sess.run([summary, main_op])
    writer.add_summary(s, global_step=epoch)

0
1
2
3
4
5
6


In [ ]:
diags = []
for _ in range(4000):
    d = np.sqrt(np.diag(model.PWalk.sigma.eval()))
    diags.append(d)
diags = np.mean(diags, axis=0)

In [ ]:
diags

In [ ]:
wsigmas = tf.sqrt(tf.diag_part(model.PWalk.sigma))

In [ ]:
wsigmas

In [ ]:
global_post=global_inf.output[0]

In [ ]:
ss = []
for _ in range(1000):
    ss.append(wsigmas.eval())
ss = np.array(ss)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
ss.std(axis=0)

In [ ]:
sns.kdeplot(ss[:,4], ss[:,1])
plt.show()